In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from torch.utils.data import DataLoader, Dataset, random_split
import torch
import torch.nn as nn
from torchvision.io import read_image
import torchvision.transforms as transforms
from torchvision import datasets
from torch import optim
import os

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
transforms = transforms.Compose([
    transforms.ToTensor()
])

In [5]:
class CustomDataset(Dataset):
    def __init__(self, labels_file, imgs_dir, transform=None, target_transform=None):
        self.labels = pd.read_csv(labels_file)
        self.imgs_dir = imgs_dir
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_path = os.path.join(self.imgs_dir, self.labels.iloc[idx, 0])
        image = torch.tensor(read_image(img_path), dtype=torch.float32)
        label = torch.tensor(self.labels.iloc[idx, 1], dtype=torch.int64)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [16]:
CustomDataset('data/Train/Train.csv', 'data/Train/', transforms).__getitem__(0)

C:\Users\Omar\AppData\Local\Temp\ipykernel_5564\94432484.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(read_image(img_path), dtype=torch.float32)


TypeError: new(): invalid data type 'str'

In [15]:
df = pd.read_csv('data/Train/Train.csv')
df.shape

(1569, 2)